In [ ]:
from hyper_embed import hyperclovaEmbedding
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.embeddings.base import Embeddings


embedding_instance = hyperclovaEmbedding(
    host='clovastudio.apigw.ntruss.com',
    api_key='your_api_key',
    api_key_primary_val='your_api_key_primary_val',
    request_id='your_request_id',
)

embeddings = []
for doc in docs:
    request_data = {
        "text": doc.page_content 
    }
    response_embedding = embedding_instance.execute(request_data)
    embeddings.append(response_embedding)


documents = [Document(page_content=doc.page_content, metadata=doc.metadata) for doc in docs]

class PrecomputedEmbeddings(Embeddings):
    def __init__(self, embeddings):
        self._embeddings = embeddings
        
    def embed_documents(self, texts):
        return self._embeddings
    
    def embed_query(self, text):
        return self._embeddings[0]


embedding_func = PrecomputedEmbeddings(embeddings)


vectorstore = FAISS.from_documents(documents, embedding_func)


vectorstore.save_local("faiss_index")

query = "넣고싶은 내용을 넣으세요"

k = 5  
search_results = vectorstore.similarity_search_with_score(query, k=k)

for doc, score in search_results:
    print(f"Score: {score}")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    print("---")